In this notebook, we load each HR image of a dataset and generate its bicubic downsampled version. This is used for the dataloader later. 

We use https://github.com/sanghyun-son/bicubic_pytorch/blob/master/core.py to do matlab like bicubic interpolation. 

In [ ]:
import matplotlib.pyplot as plt 
import core
import imageio 
import glob
import torch
import os


scale = 4 
dataset_name = "DIV2K"

## your directory which contains DIV2K 
hr_folder = "path/to/_train_HR/"
lr_folder = "save path/to/_train_LR_bicubic/X4" 


if not os.path.exists(lr_folder):
    os.makedirs(lr_folder)
    print("Created LR directory.")
    

print(hr_folder,"\n"*5,lr_folder)


In [ ]:
# For each HR image in the dataset. Load it and generate the LR by downsampling it.

hr_img_paths = glob.glob(hr_folder + "*")
for count,hr_img_path in enumerate(hr_img_paths):
    print("{}/{}".format(count,len(hr_img_paths)))
    print("Reading HR: ", hr_img_path.split("/")[-1])

    hr_img = imageio.imread(hr_img_path)
    #assert (hr_img.min() == 0 and hr_img.max() == 255)
    assert hr_img.dtype  == 'uint8'
    
    # We support 2, 3, and 4-dim Tensors
    # (H x W, C x H x W, and B x C x H x W, respectively).
    hr_img = torch.tensor(hr_img).permute(2,0,1).unsqueeze(0)
    lr_img = core.imresize(hr_img, scale=1/scale)
    # convert to numpy, astype uint8, and put channels last.
    lr_img = lr_img.squeeze(0).numpy().astype("uint8").transpose(1,2,0)
    
    # 0001 .png
    img_file = hr_img_path.split("/")[-1].split(".")[0]
    lr_img_file = img_file + "x" + str(scale) + ".png"
    lr_img_path = lr_folder + lr_img_file
    
    
    print("Writing LR: ", lr_img_path.split("/")[-1])
    print("---" * 3 )
    imageio.imwrite(lr_img_path,lr_img)
    
    